In [57]:
import dare
import random
import time
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from SISA_lib import SISA

In [58]:
data = pd.read_csv("cleaned_loan.csv")
data

,Pekerjaan,Jenis Pinjaman,Plafon (Rp.),Kondisi Joint Income,Kondisi Pisah Harta,Usia,Hasil IDEB SLIK OJK,Tujuan Pemilikan Properti,Lama bekerja,Penghasilan,...,Down Payment,Asset lainnya yang dimiliki,Bidang Usaha,Ukuran Perusahaan,Kondisi Sertifikat Agunan Utama,Jenis Bangunan Utama,Kondisi Properti Utama,Keputusan,Kelayakan Dokumen Pendukung,Keputusan Akhir
0,KARYAWAN,"PEMBELIAN BEKAS / SECONDARY, RUMAH TINGGAL","Plafon (< 514,750,000)",TIDAK JOINT INCOME,TIDAK PISAH HARTA,21 - 35,Data Tidak Tersedia,Rumah Tinggal (Rumah ke 1),"≥ 2, < 5",SINGLE : ≥RP 10 JUTA,...,>= 15%,DEPOSITO/INVES/EMAS ≥ LIMIT KREDIT,"Keuangan, asuransi, perbankan",Cakupan area : nasional,"SHM, SERTIPIKAT PECAH",Rumah Tinggal,READY STOK,GREEN,LAYAK,Diterima
1,PENGUSAHA,"PEMBELIAN BEKAS / SECONDARY, RUMAH TINGGAL","Plafon (> 1,340,960,436)",TIDAK JOINT INCOME,PISAH HARTA,36 - 45,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),≥ 5,MENIKAH : ≥RP 20 JUTA,...,>= 15%,DEPOSITO/INVES/EMAS ≥ LIMIT KREDIT,Lainnya,Cakupan area : nasional,"SHM, SERTIPIKAT PECAH",Rumah Tinggal,READY STOK,GREEN,LAYAK,Diterima
2,KARYAWAN,"TAKE OVER + TOP UP KPR, RUMAH TINGGAL","Plafon (> 1,340,960,436)",JOINT INCOME,TIDAK PISAH HARTA,46 - 55,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),≥ 5,MENIKAH : ≥RP 20 JUTA,...,>= 15%,PROPERTI/KENDARAAN ≥ LIMIT KREDIT,KLINIK AKUPUNTIR,CAKUPAN AREA : LOKAL,"SHM, SERTIPIKAT PECAH",Rumah Tinggal,SEDANG DIHUNI,GREEN,LAYAK,Diterima
3,PENGUSAHA,"PEMBELIAN BEKAS / SECONDARY, RUMAH TINGGAL","Plafon (894,632,536 - 1,340,960,436)",JOINT INCOME,TIDAK PISAH HARTA,36 - 45,"Kol 1, Tidak pernah menunggak",RUMAH TINGGAL (RUMAH KE 2/3),≥ 5,MENIKAH : ≥RP 20 JUTA,...,>= 15%,PROPERTI/KENDARAAN < 100% LIMIT KREDIT,PERDAGANGAN,CAKUPAN AREA : LOKAL,"SHM, SERTIPIKAT PECAH",RUKO/RUKAN,READY STOK,GREEN,LAYAK,Diterima
4,KARYAWAN,"PEMBELIAN BEKAS / SECONDARY, RUMAH TINGGAL","Plafon (< 514,750,000)",JOINT INCOME,TIDAK PISAH HARTA,21 - 35,Data Tidak Tersedia,Rumah Tinggal (Rumah ke 1),"≥ 2, < 5","SINGLE : ≥RP 6 JUTA, < RP. 10 JUTA",...,>= 15%,KAS DAN NON KAS ≥ LIMIT KREDIT,COLOUR SALON,CAKUPAN AREA : LOKAL,"SHGB, SERTIPIKAT PECAH",Rumah Tinggal,SEDANG DIHUNI,YELLOW,LAYAK,Diterima
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,KARYAWAN,"Pembelian Baru / Primary, Rumah Tinggal > Tipe 70","Plafon (> 1,340,960,436)",TIDAK JOINT INCOME,TIDAK PISAH HARTA,21 - 35,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),"≥ 2, < 5",SINGLE : ≥RP 20 JUTA,...,>= 15%,Properti/kendaraan < 50% plafon,"INFORMASI, TELEKOMUNIKASI DAN KOMPUTER",Public listed,MOU - Sertipikat Induk,Rumah Tinggal,Indent,YELLOW,TIDAK,Ditolak
196,KARYAWAN,"Pembelian Baru / Primary, Rumah Tinggal Tipe 2...","Plafon (< 514,750,000)",JOINT INCOME,TIDAK PISAH HARTA,21 - 35,Data Tidak Tersedia,Rumah Tinggal (Rumah ke 1),≥ 5,"MENIKAH : ≥RP 10 JUTA, <RP 20 JUTA",...,>= 15%,DEPOSITO/INVES/EMAS < 50% plafon,Lainnya,Multinasional,"SHGB, SERTIPIKAT PECAH",Rumah Tinggal,Indent,GREEN,TIDAK,Ditolak
197,KARYAWAN,"Pembelian Baru / Primary, Rumah Tinggal Tipe 2...","Plafon (< 514,750,000)",TIDAK JOINT INCOME,TIDAK PISAH HARTA,21 - 35,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),"≥ 2, < 5","SINGLE : ≥RP 6 JUTA, < RP. 10 JUTA",...,>= 15%,DEPOSITO/INVES/EMAS < 50% plafon,Lainnya,CAKUPAN AREA : LOKAL,MOU - Sertipikat Induk,Rumah Tinggal,Indent,YELLOW,TIDAK,Ditolak
198,PENGUSAHA,"Pembelian Baru / Primary, Ruko / Rukan","Plafon (> 1,340,960,436)",TIDAK JOINT INCOME,PISAH HARTA,56 - 65,"Kol 1, Tidak pernah menunggak",Rumah Tinggal (Rumah ke 1),≥ 5,MENIKAH : ≥RP 20 JUTA,...,>= 15%,null < 50% plafon,Lainnya,Multinasional,MOU - Sertipikat Induk,RUKO/RUKAN,Indent,GREEN,TIDAK,Ditolak


In [59]:
labelEncoder = {}
for col in data.columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    labelEncoder[col] = le
data

,Pekerjaan,Jenis Pinjaman,Plafon (Rp.),Kondisi Joint Income,Kondisi Pisah Harta,Usia,Hasil IDEB SLIK OJK,Tujuan Pemilikan Properti,Lama bekerja,Penghasilan,...,Down Payment,Asset lainnya yang dimiliki,Bidang Usaha,Ukuran Perusahaan,Kondisi Sertifikat Agunan Utama,Jenis Bangunan Utama,Kondisi Properti Utama,Keputusan,Kelayakan Dokumen Pendukung,Keputusan Akhir
0,0,1,2,1,1,0,0,2,1,5,...,1,2,14,1,6,2,1,0,0,0
1,1,1,3,1,0,1,8,2,2,3,...,1,2,15,1,6,2,1,0,0,0
2,0,9,3,0,1,2,8,2,2,3,...,1,9,12,0,6,2,2,0,0,0
3,1,1,1,0,1,1,8,1,2,3,...,1,7,19,0,6,1,1,0,0,0
4,0,1,2,0,1,0,0,2,1,8,...,1,5,2,0,3,2,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,6,3,1,1,0,8,2,1,7,...,1,10,6,4,0,2,0,2,1,1
196,0,7,2,0,1,0,0,2,2,2,...,1,1,15,3,3,2,0,0,1,1
197,0,7,2,1,1,0,8,2,1,8,...,1,1,15,0,0,2,0,2,1,1
198,1,5,3,1,0,3,8,2,2,3,...,1,12,15,3,0,1,0,0,1,1


In [60]:
X = data.drop(["Keputusan Akhir"], axis=1)
y = data["Keputusan Akhir"]

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [62]:
X_train_base = X_train.copy()
y_train_base = y_train.copy()
X_test_base = X_test.copy()

In [63]:
#SISA train and test dataset formats
X_train_SISA = X_train.values
y_train_SISA = y_train.values
X_test_SISA = X_test.values

In [64]:
#DaRE Forest train dataset
X_train_dare = np.array(X_train.values.tolist())
y_train_dare = np.array(y_train.values.tolist())
X_test_dare = np.array(X_test.values.tolist())

In [65]:
#make model for each machine learning model

In [66]:
#Base Model using XGBoost
rf_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
rf_model.fit(X_train_base, y_train_base)

RandomForestClassifier(max_depth=5, random_state=1)

In [67]:
#predict using XGBoost model
y_pred_xgb = rf_model.predict(X_test_base)
accuracy_xgb = accuracy_score(y_pred_xgb, y_test)
print(f"Akurasi Model XGBOOST : {accuracy_xgb}")

Akurasi Model XGBOOST : 0.875


In [68]:
#SISA Model Using 2 shards
sisa = SISA(2)
sisa.train(X_train_SISA, y_train_SISA)

In [69]:
#predict using SISA Model
y_pred_SISA = sisa.predict(X_test_SISA)
accuracy_SISA = accuracy_score(y_pred_SISA, y_test)
print(f"Akurasi Model SISA : {accuracy_SISA}")

Akurasi Model SISA : 0.95


In [70]:
#DaRE Forest Model
dafo = dare.Forest(n_estimators=100, max_depth=5, k = 5, topd = 0, random_state=1)
dafo.fit(X_train_dare, y_train_dare)

In [71]:
#Predict using DaRE Forest Model
y_pred_dare = dafo.predict(X_test_dare)
accuracy_dare = accuracy_score(y_pred_dare, y_test)
print(f"Akurasi Model DaRE Forest : {accuracy_dare}")

Akurasi Model DaRE Forest : 0.95


In [72]:
deleted_data = np.array([X_train_dare[100]])
dafo.predict_proba(deleted_data)

array([[0.07352766, 0.92647234]])

#### Melakukan proses penghapusan secara random untuk masing-masing model

In [73]:
#Create dataFrame for each model
base_model = pd.DataFrame(columns=['Iteration', 'Accuracy', 'Time'])
SISA_model = pd.DataFrame(columns=['Iteration', 'Accuracy', 'Time'])
dare_model = pd.DataFrame(columns=['Iteration', 'Accuracy', 'Time'])

In [74]:
#Create function to delete base model data
def delete_base_data(X_train_base, y_train_base, index): 
    X_data = X_train.iloc[index].tolist()
    data_to_delete = X_train_base.apply(lambda row: all(elem in row.values for elem in X_data), axis=1)
    X_train_base = X_train_base[~data_to_delete]
    y_train_base = y_train_base[~data_to_delete]
    return X_train_base, y_train_base

In [75]:
#create variabel for process
deleted_index = []
iteration = 1
max_random_number = len(X_train) - 1

In [76]:
while len(deleted_index) < max_random_number:
    #get new index
    new_index = False
    while new_index == False:
        index = random.randint(0, max_random_number - 1)
        if index not in deleted_index:
            deleted_index.append(index)
            break

    #Base Model with decreasing dataset
    try:
        X_train_base, y_train_base = delete_base_data(X_train_base, y_train_base,index)
        start_time_base = time.time()
        rf_model.fit(X_train_base, y_train_base)
        training_time_base = time.time() - start_time_base

        #prediction using base model with new data
        y_pred_xgb = rf_model.predict(X_test_base)
        accuracy_xgb = accuracy_score(y_pred_xgb, y_test)

        #add to dataframe
        base_model = pd.concat([base_model, pd.DataFrame([{'Iteration': iteration,'Accuracy': accuracy_xgb, "Time" : training_time_base}])], ignore_index=True)
    except:
        print(f"Penghapusan data mencapai maksimal")
        break

    #delete elemen by sisa
    start_time_sisa = time.time()
    sisa.delete([X_train_SISA[index]])
    training_time_SISA = time.time() - start_time_sisa

    #prediction using model with deleted data
    y_pred_SISA = sisa.predict(X_test_SISA)
    accuracy_SISA = accuracy_score(y_pred_SISA, y_test)

    #add to dataframe
    SISA_model = pd.concat([SISA_model, pd.DataFrame([{'Iteration': iteration,'Accuracy': accuracy_SISA, "Time" : training_time_SISA}])], ignore_index=True)
    
    #delete elemen by dareForest
    start_time_dare = time.time()
    dafo.delete(index)
    training_time_dare = time.time() - start_time_dare

    #prediction using model with deleted data
    y_pred_dare = dafo.predict(X_test_dare)
    accuracy_dare = accuracy_score(y_pred_dare, y_test)

    #add to dataframe
    dare_model = pd.concat([dare_model, pd.DataFrame([{'Iteration': iteration,'Accuracy': accuracy_dare, "Time" : training_time_dare}])], ignore_index=True)
    
    #add iteration
    iteration += 1

Error occurred: Invalid classes inferred from unique values of `y`.  Expected: [0], got [1]
Penghapusan data mencapai maksimal


In [77]:
base_model

,Iteration,Accuracy,Time
0,1,0.950,0.082988
1,2,0.925,0.090798
2,3,0.950,0.097240
3,4,0.925,0.104217
4,5,0.950,0.090202
...,...,...,...
152,153,0.525,0.117598
153,154,0.525,0.086937
154,155,0.525,0.097296
155,156,0.525,0.088300


In [78]:
SISA_model

,Iteration,Accuracy,Time
0,1,0.925,0.017715
1,2,0.925,0.013796
2,3,0.925,0.027736
3,4,0.925,0.039537
4,5,0.925,0.014260
...,...,...,...
152,153,0.475,0.017407
153,154,0.525,0.016695
154,155,0.525,0.019276
155,156,0.525,0.000000


In [79]:
dare_model

,Iteration,Accuracy,Time
0,1,0.950,0.002978
1,2,0.950,0.006976
2,3,0.950,0.002427
3,4,0.950,0.006897
4,5,0.950,0.000000
...,...,...,...
152,153,0.850,0.006643
153,154,0.625,0.001000
154,155,0.800,0.001000
155,156,0.775,0.001000


In [80]:
base_model.to_csv("base_exp.csv", index=False)

In [81]:
SISA_model.to_csv("sisa_exp.csv", index=False)

In [82]:
dare_model.to_csv("dare_exp.csv", index=False)

In [83]:
dare_model.iloc[130:]

,Iteration,Accuracy,Time
130,131,0.875,0.003616
131,132,0.875,0.001999
132,133,0.875,0.006935
133,134,0.875,0.002484
134,135,0.875,0.003007
135,136,0.875,0.000000
136,137,0.875,0.006990
137,138,0.900,0.006939
138,139,0.925,0.000000
139,140,0.925,0.006957
